<a href="https://colab.research.google.com/github/Narri95/NEXT_WORD_PREDICTION/blob/main/Next_word_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [3]:
from google.colab import files
uploaded=files.upload()

Saving pg.txt to pg.txt


In [4]:
file=open('pg.txt', 'r', encoding='utf8')

#store file in list
lines=[]
for i in file:
  lines.append(i)

data=""
for i in lines:
  data=' '.join(lines)

## replacing unnecessary stuff with space
data=data.replace('\n', '').replace('\r', '').replace('\ufeff','').replace('"','').replace('"','')

##remove unnecessory spaces
data=data.split()
data=' '. join(data)
data[:500]

'The Project Gutenberg eBook of Pride and prejudice, by Jane Austen This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using th'

In [5]:
##Applying tokenization
tokenizer=Tokenizer()
tokenizer.fit_on_texts([data])
#saving the tokenizer for predict funtiion
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data= tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[1, 182, 163, 1030, 3, 305, 4, 975, 30, 73, 726, 42, 1030, 24, 22]

In [6]:
 len(sequence_data)

133008

In [7]:
vocab_size=len(tokenizer.word_index)+1
print(vocab_size)

7545


In [8]:
sequences=[]

for i in range(3, len(sequence_data)):
  words=sequence_data[i-3:i+1]
  sequences.append(words)
print("the length of sequences are:", len(sequences))
sequences=np.array(sequences)
sequences[:10]

the length of sequences are: 133005


array([[   1,  182,  163, 1030],
       [ 182,  163, 1030,    3],
       [ 163, 1030,    3,  305],
       [1030,    3,  305,    4],
       [   3,  305,    4,  975],
       [ 305,    4,  975,   30],
       [   4,  975,   30,   73],
       [ 975,   30,   73,  726],
       [  30,   73,  726,   42],
       [  73,  726,   42, 1030]])

In [9]:
X=[]
y=[]

for i in sequences:
  X.append(i[0:3])
  y.append(i[3])

X=np.array(X)
y=np.array(y)

In [10]:
print('data:', X[:10])
print('Response:',y[:10])

data: [[   1  182  163]
 [ 182  163 1030]
 [ 163 1030    3]
 [1030    3  305]
 [   3  305    4]
 [ 305    4  975]
 [   4  975   30]
 [ 975   30   73]
 [  30   73  726]
 [  73  726   42]]
Response: [1030    3  305    4  975   30   73  726   42 1030]


In [11]:
 y=to_categorical(y, num_classes=vocab_size)
 y[:8]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [12]:
## creating model

model=Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [13]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             75450     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 7545)              7552545   
                                                                 
Total params: 20,676,995
Trainable params: 20,676,995
Non-trainable params: 0
_________________________________________________________________


In [14]:
##build our model

from tensorflow.keras.callbacks import ModelCheckpoint


checkpoint=ModelCheckpoint('next_words.h5', monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X,y, epochs=10, batch_size=64, callbacks=[checkpoint])

Epoch 1/10
2079/2079 [==============================] - ETA: 0s - loss: 6.2762
Epoch 1: loss improved from inf to 6.27625, saving model to next_words.h5
2079/2079 [==============================] - 46s 18ms/step - loss: 6.2762
Epoch 2/10
2079/2079 [==============================] - ETA: 0s - loss: 5.6432
Epoch 2: loss improved from 6.27625 to 5.64316, saving model to next_words.h5
2079/2079 [==============================] - 31s 15ms/step - loss: 5.6432
Epoch 3/10
2079/2079 [==============================] - ETA: 0s - loss: 5.2868
Epoch 3: loss improved from 5.64316 to 5.28676, saving model to next_words.h5
2079/2079 [==============================] - 31s 15ms/step - loss: 5.2868
Epoch 4/10
2078/2079 [============================>.] - ETA: 0s - loss: 5.0364
Epoch 4: loss improved from 5.28676 to 5.03627, saving model to next_words.h5
2079/2079 [==============================] - 30s 15ms/step - loss: 5.0363
Epoch 5/10
2078/2079 [============================>.] - ETA: 0s - loss: 4.8191
E

In [18]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

def Predict_Next_Words(model, tokenizer, text, num_words=5):
    sequence = tokenizer.texts_to_sequences([text])
    sequences = np.array(sequence)
    preds = np.argsort(model.predict(sequences), axis=1)
    predicted_words = []

    for pred in preds[0, -num_words:]:
        for key, value in tokenizer.word_index.items():
            if value == pred:
                predicted_words.append(key)
                break

    return predicted_words

model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

predicted_words = Predict_Next_Words(model, tokenizer, 'input_text', num_words=5)
print(predicted_words)


1/1 [==============================] - 1s 932ms/step
['but', 'to', 'as', '”', 'and']
